<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# Pyspark TensorFlow Inference

### Classification using Keras Preprocessing Layers

In this notebook, we demonstrate distributed inference using Keras preprocessing layers to classify structured data.  
From: https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers

Note that cuFFT/cuDNN/cuBLAS registration errors are expected (as of `tf=2.17.0`) and will not affect behavior, as noted in [this issue.](https://github.com/tensorflow/tensorflow/issues/62075)  

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

2025-01-06 21:14:44.313899: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-06 21:14:44.321948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-06 21:14:44.330652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-06 21:14:44.333277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-06 21:14:44.340133: I tensorflow/core/platform/cpu_feature_guar

In [2]:
os.mkdir('models') if not os.path.exists('models') else None

In [3]:
device = 0 if tf.config.list_physical_devices('GPU') else -1

In [4]:
print(tf.__version__)

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2.17.0


#### Download dataset

Download the PetFinder dataset from Kaggle, which where each row describes a pet and the goal is to predict adoption speed.

In [5]:
import pathlib
import os
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'

data_dir = tf.keras.utils.get_file('petfinder_mini.zip', dataset_url, extract=True, cache_dir='.')
data_dir = pathlib.Path(data_dir)
try:
    # pet-finder-mini might be under a parent a directory petfinder_mini_extracted. Check if this is the case:
    dataset = os.path.join(os.path.dirname(data_dir), 'petfinder_mini_extracted/petfinder-mini/petfinder-mini.csv')
    dataframe = pd.read_csv(dataset)
except:
    dataset = os.path.join(os.path.dirname(data_dir), 'petfinder-mini/petfinder-mini.csv')
    dataframe = pd.read_csv(dataset)

1668792/1668792 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


### Prepare dataset

In [7]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicates
# a pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop unused features.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

In [8]:
train, val, test = np.split(dataframe.sample(frac=1), [int(0.8*len(dataframe)), int(0.9*len(dataframe))])

/home/rishic/anaconda3/envs/spark-dl-tf/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

9229 training examples
1154 validation examples
1154 test examples


Create an input pipeline which converts each dataset into a tf.data.Dataset with shuffling and batching.

In [10]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key: value.to_numpy()[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

Check the format of the data returned by the pipeline:

In [11]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

2025-01-06 21:14:45.776288: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42175 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6


In [12]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch )

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
A batch of ages: tf.Tensor(
[[ 2]
 [ 2]
 [12]
 [ 3]
 [ 2]], shape=(5, 1), dtype=int64)
A batch of targets: tf.Tensor([1 1 1 1 1], shape=(5,), dtype=int64)


2025-01-06 21:14:45.853063: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Apply Keras preprocessing layers

We'll define a normalization layer for numeric features, and a category encoding for categorical features.

In [13]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [14]:
photo_count_col = train_features['PhotoAmt']
layer = get_normalization_layer('PhotoAmt', train_ds)
layer(photo_count_col)

2025-01-06 21:14:47.344194: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.43390083],
       [ 0.43390083],
       [ 2.004037  ],
       [ 1.0619552 ],
       [-0.822208  ]], dtype=float32)>

In [15]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    # Create a layer that turns strings into integer indices.
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    # Otherwise, create a layer that turns integer values into integer indices.
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))

In [16]:
test_type_col = train_features['Type']
test_type_layer = get_category_encoding_layer(name='Type',
                                              dataset=train_ds,
                                              dtype='string')
test_type_layer(test_type_col)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)>

In [17]:
test_age_col = train_features['Age']
test_age_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
test_age_layer(test_age_col)

2025-01-06 21:14:48.950627: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

### Preprocess selected features

Apply the preprocessing utility functions defined earlier. Add all the feature inputs to a list.


In [18]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [19]:
all_inputs = {}
encoded_features = []

# Numerical features.
for header in ['PhotoAmt', 'Fee']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs[header] = numeric_col
    encoded_features.append(encoded_numeric_col)

In [20]:
age_col = tf.keras.Input(shape=(1,), name='Age', dtype='int64')

encoding_layer = get_category_encoding_layer(name='Age',
                                             dataset=train_ds,
                                             dtype='int64',
                                             max_tokens=5)
encoded_age_col = encoding_layer(age_col)
all_inputs['Age'] = age_col
encoded_features.append(encoded_age_col)

In [21]:
categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']

for header in categorical_cols:
    categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
    encoding_layer = get_category_encoding_layer(name=header,
                                                dataset=train_ds,
                                                dtype='string',
                                                max_tokens=5)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs[header] = categorical_col
    encoded_features.append(encoded_categorical_col)

2025-01-06 21:14:49.223773: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-01-06 21:14:49.659659: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Create, compile, and train model

In [22]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)

In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"],
              run_eagerly=True)

In [24]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4366 - loss: 0.6702 - val_accuracy: 0.7062 - val_loss: 0.5727
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6428 - loss: 0.5924 - val_accuracy: 0.7236 - val_loss: 0.5529
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6820 - loss: 0.5629 - val_accuracy: 0.7418 - val_loss: 0.5404
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6850 - loss: 0.5624 - val_accuracy: 0.7400 - val_loss: 0.5313
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6953 - loss: 0.5633 - val_accuracy: 0.7374 - val_loss: 0.5253
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6982 - loss: 0.5516 - val_accuracy: 0.7392 - val_loss: 0.5199
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7049 - loss: 0.5438 - val_accuracy: 0.7444 - val_loss: 0.5166
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7109 - loss: 0.5362 - val_accuracy: 0.7435 - v

In [25]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7812 - loss: 0.4536

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7601 - loss: 0.4909 
Accuracy 0.7478336095809937


### Save and reload model

Demonstrate saving the trained model and reloading it for inference.

In [26]:
model.save('models/my_pet_classifier.keras')

In [27]:
reloaded_model = tf.keras.models.load_model('models/my_pet_classifier.keras')

In [28]:
sample = {
    'Type': 'Cat',
    'Age': 3,
    'Breed1': 'Tabby',
    'Gender': 'Male',
    'Color1': 'Black',
    'Color2': 'White',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    'Vaccinated': 'No',
    'Sterilized': 'No',
    'Health': 'Healthy',
    'Fee': 100,
    'PhotoAmt': 2,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = reloaded_model.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print(
    "This particular pet had a %.1f percent probability "
    "of getting adopted." % (100 * prob)
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
This particular pet had a 78.4 percent probability of getting adopted.


## PySpark

In [29]:
from pyspark.sql.functions import col, struct, pandas_udf
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkConf
import json
import pandas as pd

Check the cluster environment to handle any platform-specific Spark configurations.

In [30]:
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [31]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
        # Point PyTriton to correct libpython3.11.so:
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_env}/lib:{conda_env}/lib/python3.11/site-packages/nvidia_pytriton.libs:$LD_LIBRARY_PATH")
        source = "/usr/lib/x86_64-linux-gnu/libstdc++.so.6"
        target = f"{conda_env}/lib/libstdc++.so.6"
        try:
            if os.path.islink(target) or os.path.exists(target):
                os.remove(target)
            os.symlink(source, target)
        except OSError as e:
            print(f"Error creating symlink: {e}")
    elif on_dataproc:
        # Point PyTriton to correct libpython3.11.so:
        conda_lib_path="/opt/conda/miniconda3/lib"
        conf.set("spark.executorEnv.LD_LIBRARY_PATH", f"{conda_lib_path}:$LD_LIBRARY_PATH") 

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/01/06 21:14:57 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/01/06 21:14:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/06 21:14:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Create PySpark DataFrame

In [32]:
df = spark.createDataFrame(dataframe).repartition(8)

In [33]:
data_path = "spark-dl-datasets/petfinder-mini"
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/spark-dl-datasets")
    data_path = "dbfs:/FileStore/" + data_path

df.write.mode("overwrite").parquet(data_path)

#### Load and preprocess DataFrame

In [34]:
df = spark.read.parquet(data_path).cache()

In [35]:
columns = df.columns
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']


In [36]:
# remove label column
columns.remove("target")
print(columns)

['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']


In [37]:
df.show(5)

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized| Health|Fee|PhotoAmt|target|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+-------+---+--------+------+
| Dog|  3|         Mixed Breed|  Male| Black|No Color|       Small|   Medium|  Not Sure|  Not Sure|Healthy|  0|       2|     0|
| Dog|  9|         Mixed Breed|  Male|  Gray|No Color|      Medium|    Short|  Not Sure|        No|Healthy|  0|       4|     1|
| Cat|  4| Domestic Short Hair|  Male| Black|    Gray|      Medium|    Short|  Not Sure|  Not Sure|Healthy|  0|       4|     1|
| Cat|  6| Domestic Short Hair|  Male|Yellow|   White|      Medium|    Short|        No|        No|Healthy|  0|       3|     1|
| Cat|  6|Domestic Medium Hair|  Male|  Gray|No Color|       Small|   Medium|        No|        No|Healt

## Inference using Spark DL API

Distributed inference using the PySpark [predict_batch_udf](https://spark.apache.org/docs/3.4.0/api/python/reference/api/pyspark.ml.functions.predict_batch_udf.html#pyspark.ml.functions.predict_batch_udf):

- predict_batch_fn uses Tensorflow APIs to load the model and return a predict function which operates on numpy arrays 
- predict_batch_udf will convert the Spark DataFrame columns into numpy input batches for the predict function

In [38]:
# get absolute path to model
model_path = "{}/models/my_pet_classifier.keras".format(os.getcwd())

# For cloud environments, copy the model to the distributed file system.
if on_databricks:
    import shutil
    dbutils.fs.mkdirs("/FileStore/spark-dl-models")
    dbfs_model_path = "/dbfs/FileStore/spark-dl-models/my_pet_classifier.keras"
    shutil.copy(model_path, dbfs_model_path)
    model_path = dbfs_model_path

In [39]:
def predict_batch_fn():
    import tensorflow as tf
    import pandas as pd
    
    # Enable GPU memory growth to avoid CUDA OOM
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = tf.keras.models.load_model(model_path)

    def predict(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        inputs = {
            "Type": t,
            "Age": a,
            "Breed1": b,
            "Gender": g,
            "Color1": c1,
            "Color2": c2,
            "MaturitySize": m,
            "FurLength": f,
            "Vaccinated": v,
            "Sterilized": s,
            "Health": h,
            "Fee": fee,
            "PhotoAmt": p
        }
        # return model.predict(inputs)
        return pd.Series(np.squeeze(model.predict(inputs)))

    return predict

In [40]:
# need to pass the list of columns into the model_udf
classify = predict_batch_udf(predict_batch_fn,
                             return_type=FloatType(),
                             batch_size=100)

In [41]:
%%time
preds = df.withColumn("preds", classify(struct(*columns)))
results = preds.collect()

25/01/06 21:15:01 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 17.1 ms, sys: 7.86 ms, total: 25 ms
Wall time: 4.81 s


In [42]:
%%time
preds = df.withColumn("preds", classify(*columns))
results = preds.collect()

CPU times: user 79.2 ms, sys: 16.6 ms, total: 95.8 ms
Wall time: 1.59 s


In [43]:
%%time
preds = df.withColumn("preds", classify(*[col(c) for c in columns]))
results = preds.collect()

CPU times: user 19.5 ms, sys: 5.43 ms, total: 25 ms
Wall time: 1.51 s


In [44]:
preds.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized|      Health|Fee|PhotoAmt|target|      preds|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
| Dog|  3|         Mixed Breed|  Male| Black|No Color|       Small|   Medium|  Not Sure|  Not Sure|     Healthy|  0|       2|     0| 0.39443576|
| Dog|  9|         Mixed Breed|  Male|  Gray|No Color|      Medium|    Short|  Not Sure|        No|     Healthy|  0|       4|     1|  0.7342956|
| Cat|  4| Domestic Short Hair|  Male| Black|    Gray|      Medium|    Short|  Not Sure|  Not Sure|     Healthy|  0|       4|     1| 0.47965235|
| Cat|  6| Domestic Short Hair|  Male|Yellow|   White|      Medium|    Short|        No|        No|     Healthy|  0|       3|     

## Using Triton Inference Server
In this section, we demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-pytriton.png" alt="drawing" width="700"/>

In [45]:
from functools import partial

In [46]:
def triton_server(ports, model_path):
    import time
    import signal
    import numpy as np
    import tensorflow as tf
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing model on worker {TaskContext.get().partitionId()}.")
    # Enable GPU memory growth
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

    model = tf.keras.models.load_model(model_path)

    def decode(input_tensor):
        return tf.convert_to_tensor(np.vectorize(lambda x: x.decode('utf-8'))(input_tensor))

    def identity(input_tensor):
        return tf.convert_to_tensor(input_tensor)

    input_transforms = {
        "Type": decode,
        "Age": identity,
        "Breed1": decode,
        "Gender": decode,
        "Color1": decode,
        "Color2": decode,
        "MaturitySize": decode,
        "FurLength": decode,
        "Vaccinated": decode,
        "Sterilized": decode,
        "Health": decode,
        "Fee": identity,
        "PhotoAmt": identity
    }

    @batch
    def _infer_fn(**inputs):
        decoded_inputs = {k: input_transforms[k](v) for k, v in inputs.items()}
        print(f"SERVER: Received batch of size {len(decoded_inputs['Type'])}.")
        return {
            "preds": model.predict(decoded_inputs)
        }

    workspace_path = f"triton_workspace_{time.strftime('%Y%m%d_%H%M%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="PetClassifier",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="Type", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Age", dtype=np.int64, shape=(-1,)),
                Tensor(name="Breed1", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Gender", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Color1", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Color2", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="MaturitySize", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="FurLength", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Vaccinated", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Sterilized", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Health", dtype=np.bytes_, shape=(-1,)),
                Tensor(name="Fee", dtype=np.int64, shape=(-1,)),
                Tensor(name="PhotoAmt", dtype=np.int64, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="preds", dtype=np.float32, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=128,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def stop_triton(signum, frame):
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

def start_triton(ports, model_name, model_path):
    import socket
    from pathlib import Path
    from multiprocessing import Process
    from pytriton.client import ModelClient

    os.chdir(Path.home())

    hostname = socket.gethostname()
    process = Process(target=triton_server, args=(ports, model_path,))
    process.start()

    client = ModelClient(f"http://localhost:{ports[0]}", model_name)
    patience = 8
    while patience > 0:
        try:
            client.wait_for_server(5)
            return [(hostname, process.pid)]
        except Exception:
            print("Waiting for server to be ready...")
            patience -= 1

    emsg = "Failure: client waited too long for server startup."
    print(emsg)
    return [(hostname, emsg)]

#### Start Triton servers

To ensure that only one Triton inference server is started per node, we use stage-level scheduling to delegate each task to a separate GPU.  

In [47]:
def _use_stage_level_scheduling(spark, rdd):

    if spark.version < "3.4.0":
        raise Exception("Stage-level scheduling is not supported in Spark < 3.4.0")

    executor_cores = spark.conf.get("spark.executor.cores")
    assert executor_cores is not None, "spark.executor.cores is not set"
    executor_gpus = spark.conf.get("spark.executor.resource.gpu.amount")
    assert executor_gpus is not None and int(executor_gpus) <= 1, "spark.executor.resource.gpu.amount must be set and <= 1"

    from pyspark.resource.profile import ResourceProfileBuilder
    from pyspark.resource.requests import TaskResourceRequests

    spark_plugins = spark.conf.get("spark.plugins", " ")
    assert spark_plugins is not None
    spark_rapids_sql_enabled = spark.conf.get("spark.rapids.sql.enabled", "true")
    assert spark_rapids_sql_enabled is not None

    task_cores = (
        int(executor_cores)
        if "com.nvidia.spark.SQLPlugin" in spark_plugins
        and "true" == spark_rapids_sql_enabled.lower()
        else (int(executor_cores) // 2) + 1
    )

    task_gpus = 1.0
    treqs = TaskResourceRequests().cpus(task_cores).resource("gpu", task_gpus)
    rp = ResourceProfileBuilder().require(treqs).build
    print(f"Reqesting stage-level resources: (cores={task_cores}, gpu={task_gpus})")

    return rdd.withResources(rp)

**Specify the number of nodes in the cluster.**  
Following the README, the example standalone cluster uses 1 node. The example Databricks/Dataproc cluster scripts use 4 nodes by default. 

In [48]:
# Change based on cluster setup
num_nodes = 1 if on_standalone else 4

In [49]:
sc = spark.sparkContext
nodeRDD = sc.parallelize(list(range(num_nodes)), num_nodes)
nodeRDD = _use_stage_level_scheduling(spark, nodeRDD)

Reqesting stage-level resources: (cores=5, gpu=1.0)


Triton occupies ports for HTTP requests, GRPC requests, and the metrics service.

In [50]:
def find_ports():
    import psutil
    
    ports = []
    conns = [conn.laddr.port for conn in psutil.net_connections(kind="inet")]
    i = 7000
    while len(ports) < 3:
        if i not in conns:
            ports.append(i)
        i += 1
    
    return ports

In [51]:
model_name = "PetClassifier"
ports = find_ports()
assert len(ports) == 3
print(f"Using ports {ports}")

Using ports [7000, 7001, 7002]


In [52]:
pids = nodeRDD.barrier().mapPartitions(lambda _: start_triton(ports, model_name, model_path)).collectAsMap()
print("Triton Server PIDs:\n", json.dumps(pids, indent=4))

Triton Server PIDs:
 {
    "cb4ae00-lcedt": 2806426
}


#### Define client function

In [53]:
url = f"http://localhost:{ports[0]}"

In [54]:
def triton_fn(url, model_name, init_timeout_s):
    import numpy as np
    from pytriton.client import ModelClient

    print(f"CLIENT: Connecting to {model_name} at {url}")

    def infer_batch(t, a, b, g, c1, c2, m, f, v, s, h, fee, p):
        
        def encode(value):
            return np.vectorize(lambda x: x.encode("utf-8"))(value).astype(np.bytes_)

        with ModelClient(url, model_name, init_timeout_s=init_timeout_s) as client:
            encoded_inputs = {
                "Type": encode(t), 
                "Age": a, 
                "Breed1": encode(b), 
                "Gender": encode(g),
                "Color1": encode(c1),
                "Color2": encode(c2),
                "MaturitySize": encode(m),
                "FurLength": encode(f),
                "Vaccinated": encode(v),
                "Sterilized": encode(s),
                "Health": encode(h),
                "Fee": fee,
                "PhotoAmt": p
            }
            result_data = client.infer_batch(**encoded_inputs)
            return result_data["preds"]
            
    return infer_batch

#### Load and preprocess DataFrame

In [55]:
df = spark.read.parquet(data_path)

In [56]:
columns = df.columns
# remove label column
columns.remove("target")

#### Run inference

In [57]:
# need to pass the list of columns into the model_udf
classify = predict_batch_udf(partial(triton_fn, url=url, model_name=model_name, init_timeout_s=600),
                             input_tensor_shapes=[[1]] * len(columns),
                             return_type=FloatType(),
                             batch_size=64)

In [58]:
%%time
preds = df.withColumn("preds", classify(struct(*columns)))
results = preds.collect()

CPU times: user 22.5 ms, sys: 5.09 ms, total: 27.6 ms
Wall time: 8.46 s


In [59]:
%%time
preds = df.withColumn("preds", classify(*columns))
results = preds.collect()

CPU times: user 14.9 ms, sys: 4.68 ms, total: 19.6 ms
Wall time: 5.81 s


In [60]:
%%time
preds = df.withColumn("preds", classify(*[col(c) for c in columns]))
results = preds.collect()

CPU times: user 87.8 ms, sys: 4.62 ms, total: 92.5 ms
Wall time: 5.96 s


In [61]:
preds.show()

+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
|Type|Age|              Breed1|Gender|Color1|  Color2|MaturitySize|FurLength|Vaccinated|Sterilized|      Health|Fee|PhotoAmt|target|      preds|
+----+---+--------------------+------+------+--------+------------+---------+----------+----------+------------+---+--------+------+-----------+
| Dog|  3|         Mixed Breed|  Male| Black|No Color|       Small|   Medium|  Not Sure|  Not Sure|     Healthy|  0|       2|     0| 0.39443576|
| Dog|  9|         Mixed Breed|  Male|  Gray|No Color|      Medium|    Short|  Not Sure|        No|     Healthy|  0|       4|     1|  0.7342956|
| Cat|  4| Domestic Short Hair|  Male| Black|    Gray|      Medium|    Short|  Not Sure|  Not Sure|     Healthy|  0|       4|     1| 0.47965235|
| Cat|  6| Domestic Short Hair|  Male|Yellow|   White|      Medium|    Short|        No|        No|     Healthy|  0|       3|     

#### Stop Triton Server on each executor

In [62]:
def stop_triton(pids):
    import os
    import socket
    import signal
    import time 
    
    hostname = socket.gethostname()
    pid = pids.get(hostname, None)
    assert pid is not None, f"Could not find pid for {hostname}"
    
    for _ in range(5):
        try:
            os.kill(pid, signal.SIGTERM)
        except OSError:
            return [True]
        time.sleep(5)

    return [False]

shutdownRDD = sc.parallelize(list(range(num_nodes)), num_nodes)
shutdownRDD = _use_stage_level_scheduling(spark, shutdownRDD)
shutdownRDD.barrier().mapPartitions(lambda _: stop_triton(pids)).collect()

Reqesting stage-level resources: (cores=5, gpu=1.0)


[True]

In [ ]:
spark.stop()